In [2]:
import mysql.connector as mysql
import pandas as pd
import re

In [3]:
CREATE_TABLES=True
POPULATE_TABLES=False
DROP_TABLES = False
PRINT_RESULTS=True
EXPORT_TO_EXCEL=True

if DROP_TABLES or CREATE_TABLES:
    CREATE_TABLES=True
    POPULATE_TABLES=True

In [4]:
def convert_to_pretty_table(cursor):
    from prettytable import PrettyTable
  
    # Specify the Column Names while initializing the Table
    column_names=[]

    for x in cursor.description:
        column_names.append(x[0])
    myTable = PrettyTable(column_names)
    
    for x in cursor:
        myTable.add_row(list(x))
    
    print(myTable)

def convert_to_dataframe(cursor):
    column_names=[]
    for x in cursor.description:
        column_names.append(x[0])
    df=pd.DataFrame(columns=column_names)
    for x in cursor:
        df.loc[len(df)]=list(x)
    return df

In [5]:
connector=mysql.connect(host='localhost', user='root', password='password', database='ClointFusion')
cursor=connector.cursor()

In [6]:
SHOW_TABLES_QUERY='SHOW TABLES'
cursor.execute(SHOW_TABLES_QUERY)
tables=cursor.fetchall()
if DROP_TABLES:
    for table in tables:
        cursor.execute("DROP TABLE {table}".format(table=table[0]))
    connector.commit()

In [7]:
if CREATE_TABLES:
    tables=["CREATE TABLE if not exists order_report(OrderID INT,sku VARCHAR(255),Order_Qty INT)"
    ,"Create TABLE if not exists sku_master(sku VARCHAR(255),Weight FLOAT)"
    ,"Create TABLE if not exists Courier_company_invoice(AWB_Code VARCHAR(255),OrderID INT,Charged_Weight FLOAT,Warehouse_Pincode INT,Customer_Pincode INT,Zone varchar(255),Type_of_Shipment varchar(255),Billing_Amount FLOAT)"
    ,"CREATE TABLE if not exists company_pincode (Warehouse_Pincode INT,Customer_Pincode INT,Zone varchar(255))","CREATE TABLE if not exists courier_rates(Zone varchar(255),type varchar(255),cond varchar(255),Charges FLOAT)"]

    for table in tables:
        print(tables.index(table)+1)
        cursor.execute(table)

if POPULATE_TABLES:
    excel_path=r"Dataset\Company X - Order Report.xlsx"
    df=pd.read_excel(excel_path)
    for row in range(df.shape[0]):
        order_id,sku,order_qty=df.iloc[row,:].to_list()
        cursor.execute("INSERT INTO order_report VALUES "+str((order_id,sku,order_qty)))
    cursor.execute("SELECT COUNT(*) FROM order_report")
    if cursor.fetchall()[0][0]!=df.shape[0]:
        print("Error")
    excel_path=r"Dataset\Company X - SKU Master.xlsx"
    df=pd.read_excel(excel_path)
    for row in range(df.shape[0]):
        sku,weight=df.iloc[row,:].to_list()
        cursor.execute("INSERT INTO sku_master VALUES "+str((sku,weight)))
    cursor.execute("SELECT COUNT(*) FROM sku_master")
    if cursor.fetchall()[0][0]!=df.shape[0]:
        print("Error")

    excel_path=r"Dataset\Courier Company - Invoice.xlsx"
    df=pd.read_excel(excel_path)
    for row in range(df.shape[0]):
        AWB_Code,OrderID,Charged_Weight,Warehouse_Pincode,Customer_Pincode,Zone,Type_of_Shipment,Billing_Amount=df.iloc[row,:].to_list()
        cursor.execute("INSERT INTO Courier_company_invoice VALUES "+str((AWB_Code,OrderID,Charged_Weight,Warehouse_Pincode,Customer_Pincode,Zone,Type_of_Shipment,Billing_Amount)))
    cursor.execute("SELECT Count(*) From Courier_company_invoice")
    if cursor.fetchall()[0][0]!=df.shape[0]:
        print("Error")

    excel_path=r"Dataset\Company X - Pincode Zones.xlsx"
    df=pd.read_excel(excel_path)
    for row in range(df.shape[0]):
        Warehouse_Pincode,Customer_Pincode,Zone=df.iloc[row,:].to_list()
        cursor.execute("INSERT INTO company_pincode VALUES "+str((Warehouse_Pincode,Customer_Pincode,Zone)))
    cursor.execute("SELECT COUNT(*) FROM company_pincode")
    if cursor.fetchall()[0][0]!=df.shape[0]:
        print("Error")

    df=pd.read_excel(r"Dataset\Courier Company - Rates.xlsx")
    df=df.transpose()
    df.rename(columns={0:'charge'},inplace=True)
    df["zone"]=None
    df["cond"]=None
    df["type"]=None
    for i in range(df.shape[0]):
        df["type"][i]=df.index[i].split('_')[0]
        df["zone"][i]=df.index[i].split('_')[1]
        df["cond"][i]=df.index[i].split('_')[2]
    for i in range(df.shape[0]):
        cursor.execute("INSERT INTO courier_rates VALUES "+str((df["zone"][i],df["type"][i],df["cond"][i],df["charge"][i])))
    cursor.execute("SELECT COUNT(*) FROM courier_rates")
    if cursor.fetchall()[0][0]!=df.shape[0]:
        print("Error")

    connector.commit()

1
2
3
4


In [ ]:
cursor.execute("""SELECT COUNT(o.OrderID) as order_report,COUNT(s.sku) AS sku_master, COUNT(c.OrderID) as Courier_company_invoice,COUNT(cp.Zone) as company_pincode,count(cr.Zone) as courier_rates 
                FROM order_report o, sku_master s, Courier_company_invoice c, company_pincode cp, courier_rates cr""")
convert_to_pretty_table(cursor)

NameError: name 'cursor' is not defined

In [ ]:
cursor.execute("SELECT * FROM order_report LIMIT 5")
convert_to_pretty_table(cursor)

In [ ]:
cursor.execute("SELECT * FROM sku_master LIMIT 5")
convert_to_pretty_table(cursor)

In [ ]:
cursor.execute("SELECT * FROM Courier_company_invoice LIMIT 5")
convert_to_pretty_table(cursor)

In [ ]:
cursor.execute("SELECT * FROM company_pincode LIMIT 5")
convert_to_pretty_table(cursor)

In [ ]:
cursor.execute("SELECT * FROM courier_rates")
convert_to_pretty_table(cursor)

In [ ]:
def get_charges(cursor):
    cursor.execute("SELECT * FROM courier_rates")
    return convert_to_dataframe(cursor)

import numpy as np
def calc_charges(calculate_charge,weight,zone):
    charges=get_charges(cursor)
    total=0.00
    
    if "forward" in calculate_charge.lower():
        fixed_charge=float(charges["Charges"].iloc[np.where((charges["cond"]=="fixed") & (charges["type"]=="fwd") & (charges["Zone"]==zone))[0]])
        additional_charge=float(charges["Charges"].iloc[np.where((charges["cond"]=="additional") & (charges["type"]=="fwd") & (charges["Zone"]==zone))[0]])
        total+=fixed_charge+additional_charge*(weight-0.5 if weight-0.5>0 else 0)

    if "rto" in calculate_charge.lower():
        fixed_charge=float(charges["Charges"].iloc[np.where((charges["cond"]=="fixed") & (charges["type"]=="rto") & (charges["Zone"]==zone))[0]])
        additional_charge=float(charges["Charges"].iloc[np.where((charges["cond"]=="additional") & (charges["type"]=="rto") & (charges["Zone"]==zone))[0]])
        total+=fixed_charge+additional_charge*(weight-0.5 if weight-0.5>0 else 0)
    return total

In [ ]:
cursor.execute("""SELECT o.sku,o.OrderID, ROUND(CEILING(s.Weight*o.Order_Qty/500)/2,1) as total_weight_slab, (s.Weight*o.Order_Qty/1000) as total_weight_actual, cp.Zone as zone_actual ,c.Zone as zone_charged, c.Type_of_Shipment, c.Billing_Amount, c.Charged_Weight
                FROM order_report o,sku_master s, Courier_company_invoice c, company_pincode cp
                WHERE o.sku=s.sku AND o.OrderID=c.OrderID AND cp.Warehouse_Pincode=c.Warehouse_Pincode AND cp.Customer_Pincode=c.Customer_Pincode """)
company_df=convert_to_dataframe(cursor)


In [ ]:
company_df["actual_charge"]=-1.00
for i in range(company_df.shape[0]):
    type_of_shipment=company_df["Type_of_Shipment"][i]
    slab=company_df["total_weight_slab"][i]
    zone=company_df["zone_actual"][i]
    company_df["actual_charge"][i]=calc_charges(type_of_shipment,slab,zone)
company_df["surcharge"]=-company_df["actual_charge"]+company_df["Billing_Amount"]

In [ ]:
company_df.head(5)

In [ ]:
overcharged=np.where(company_df["Billing_Amount"]>company_df["actual_charge"])[0]
undercharged=np.where(company_df["Billing_Amount"]<company_df["actual_charge"])[0]
correct=np.where(company_df["Billing_Amount"]==company_df["actual_charge"])[0]

In [ ]:
total_overcharged=np.sum(company_df["Billing_Amount"].iloc[overcharged]-company_df["actual_charge"].iloc[overcharged])
total_undercharged=np.sum(company_df["Billing_Amount"].iloc[undercharged]-company_df["actual_charge"].iloc[undercharged])
total_correct=np.sum(company_df["Billing_Amount"].iloc[correct])

In [ ]:
if PRINT_RESULTS:
    output_df=pd.DataFrame(columns=["Charge","Count","Cost"])
    output_df["Charge"]=["Overcharged","Undercharged","Corectly Charged"]
    output_df["Count"]=[len(overcharged),len(undercharged),len(correct)]
    output_df["Cost"]=[total_overcharged,total_undercharged,total_correct]
    print(output_df)

In [ ]:
if EXPORT_TO_EXCEL:
    company_df.to_excel(r"D:\Freelance\Dataset-Clointtab\company_df.xlsx")
    import openpyxl as op
    wb=op.load_workbook(r"D:\Freelance\Dataset-Clointtab\company_df.xlsx")
    wb.create_sheet("Summary")
    ws=wb["Summary"]
    ws.append(["Charge","Count","Value"])
    ws.append(["Overcharged",len(overcharged),total_overcharged])
    ws.append(["Undercharged",len(undercharged),total_undercharged])
    ws.append(["Correctly Charged",len(correct),total_correct])
    wb.save(r"D:\Freelance\Dataset-Clointtab\company_df.xlsx")